In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
# define helper functions
def imShow(path): 
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd darknet/
!ls /mydrive/project/

In [ ]:
# unzip the datasets and their contents so that they are now in /darknet/data/ folder
# !cp -r /content/gdrive/MyDrive/project/veriler/ ../
# !cp -r /content/gdrive/MyDrive/project/test/ ../
!cp -r /content/gdrive/MyDrive/project/veriler/ -d data/
!cp -r /content/gdrive/MyDrive/project/test/ -d data/

In [ ]:
# download cfg to google drive and change its name
!cp cfg/yolov4-tiny-custom.cfg /content/gdrive/MyDrive/project/yolov4-tiny-obj.cfg

In [ ]:
# Darknet dosyaları arasında hazır olarak bulunan config dosyasını gerekli değişiklikleri yapmak için drive'a alalım.
!cp /content/gdrive/MyDrive/project/yolov4-tiny-obj.cfg ./cfg

In [ ]:
# Drive'da oluşturduğumuz obj.names ve obg.data dosyalarımızı collab dosyalarına alalım.
!cp /content/gdrive/MyDrive/project/obj.names ./data
!cp /content/gdrive/MyDrive/project/obj.data  ./data

In [ ]:
# Drive'da oluşturduğumuz generate dosyalarını collab dosyalarına alalım.
!cp /content/gdrive/MyDrive/project/generate_train.py ./
!cp /content/gdrive/MyDrive/project/generate_test.py ./

In [ ]:
!python /content/darknet/generate_train.py
!python /content/darknet/generate_test.py

In [ ]:
# verify that the newly generated train.txt and test.txt can be seen in our darknet/data folder
!ls data/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Bu adımda önceden eğitilmiş yolov3 için kullanılmış deeplearning katmanları ağırlıklarını indiriyoruz.
#Bu adımı uygulamak zorunda değiliz ama eğitime bu ağırlıklarla başlamak eğittiğimiz modelin daha doğru çalışmasına ve eğitim süresini kısaltmaya yardımcı olacaktır.
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
# %%capture
!./darknet detector train data/obj.data cfg/yolov4-tiny-obj.cfg yolov4-tiny.conv.29 -dont_show -map

In [ ]:
# eğitimimize ait grafiğimiz.
imShow('chart.png')

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-tiny-obj.cfg /mydrive/yolov4-tiny/backup/yolov4-tiny-obj_last.weights

In [ ]:
# need to set our custom cfg to test mode 
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-tiny-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-tiny-obj.cfg
%cd ..

In [ ]:
# run your custom detector with this command (upload an image to your google drive to test, thresh flag sets accuracy that detection must be in order to show it)
!./darknet detector test data/obj.data cfg/yolov4-tiny-obj.cfg /mydrive/project/backup/yolov4-tiny-obj_last.weights /mydrive/project/test-foto/test5.jpg -thresh 0.5 
imShow('predictions.jpg')